In [3]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [4]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

City_ID     City Country                 Date      Lat       Lng  Max Temp  \
0        0  Rikitea      PF  2021-10-03 15:57:14 -23.1203 -134.9692     73.08   
1        1   Bastia      FR  2021-10-03 15:57:14  42.6667    9.3333     75.00   
2        2     Asau      RO  2021-10-03 15:57:15  46.4333   26.4000     60.40   
3        3   Dingle      PH  2021-10-03 15:57:15  10.9995  122.6711     76.21   
4        4  Qaanaaq      GL  2021-10-03 15:57:15  77.4840  -69.3632     31.89   

   Humidity  Cloudiness  Wind Speed  
0        64           6       17.31  
1        78          20       17.27  
2        67          74        2.04  
3        94         100        5.68  
4        97         100       10.04

In [5]:
# Get the datatypes of the city_data_df
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [6]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# 1. Assign the locations to an array of latitude and longitude pairs.
locations = city_data_df[['Lat', 'Lng']]
# 2. Assign the weights variable to some values.
temperatures = city_data_df['Max Temp']
# 3. Assign the figure variable to the gmaps.figure() attribute.
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
# 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in temperatures], dissipating=False, max_intensity=300, point_radius=4)
# 5. Add the heatmap layer.
fig.add_layer(heat_layer)
# 6. Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent Max Sped
locations = city_data_df[["Lat", "Lng"]]
cloudiness = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [ ]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

In [ ]:
preferred_cities_df.count()

In [26]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [29]:
# Create hotel dataframe with City, Country, Max Temp, Lat, Lng
hotel_df = preferred_cities_df[['City','Country','Max Temp','Lat','Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

City Country  Max Temp      Lat       Lng Hotel Name
1      Bastia      FR     75.00  42.6667    9.3333           
3      Dingle      PH     76.21  10.9995  122.6711           
5    Monrovia      LR     87.57   6.3005  -10.7969           
7    Bengkulu      ID     76.64  -3.8004  102.2655           
8      Solano      US     77.02  38.2333 -121.9511           
22       Biak      ID     75.83  -0.9131  122.8766           
23   Mormugao      IN     79.86  15.4000   73.8000           
27  Mount Isa      AU     82.17 -20.7333  139.5000           
28     Atuona      PF     77.14  -9.8000 -139.0333           
31   Mabaruma      GY     86.00   8.2000  -59.7833

In [34]:
# Google Maps base URL
g_baseurl = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key,
    'location': '48.8566, 2.3522'
}
# Make request and get the JSON data from the search
hotels = requests.get(g_baseurl, params=params).json

In [36]:
# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    
    #add the latitude and longitude to the location key for the params dictionary
    params['location'] = f'{lat},{lng}'
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(g_baseurl, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [40]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [38]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} Â°F</dd>
</dl>
"""

In [39]:
# Store the hotel information into the DataFrame
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]